In [1]:
import docker
d = docker.DockerClient()


In [3]:
d.containers.list()[0].attrs

{'Id': 'c9faf0f88db278ece4a4104a05eca6aebf1e72d9076ac0100a415f474e5cc8a1',
 'Created': '2023-01-11T15:47:50.899688Z',
 'Path': 'docker-entrypoint.sh',
 'Args': ['node', 'server.js'],
 'State': {'Status': 'running',
  'Running': True,
  'Paused': False,
  'Restarting': False,
  'OOMKilled': False,
  'Dead': False,
  'Pid': 1704,
  'ExitCode': 0,
  'Error': '',
  'StartedAt': '2023-04-11T19:18:01.5612439Z',
  'FinishedAt': '2023-04-11T19:17:52.8145356Z'},
 'Image': 'sha256:24ab293812c9f4efc10c6c43db32a766b7cfce4502b7df408a6e3b06ccc53bf8',
 'ResolvConfPath': '/var/lib/docker/containers/c9faf0f88db278ece4a4104a05eca6aebf1e72d9076ac0100a415f474e5cc8a1/resolv.conf',
 'HostnamePath': '/var/lib/docker/containers/c9faf0f88db278ece4a4104a05eca6aebf1e72d9076ac0100a415f474e5cc8a1/hostname',
 'HostsPath': '/var/lib/docker/containers/c9faf0f88db278ece4a4104a05eca6aebf1e72d9076ac0100a415f474e5cc8a1/hosts',
 'LogPath': '/var/lib/docker/containers/c9faf0f88db278ece4a4104a05eca6aebf1e72d9076ac0100a415f4

In [39]:
print(
    next(filter(lambda x:x.name == "backend", d.containers.list())).exec_run("ls -la  /var/lib/docker/ ").output.decode()
)

ls: cannot access '/var/lib/docker/': No such file or directory



In [50]:
import shlex
import subprocess


def generator():
    buffer = []
    process = subprocess.Popen(shlex.split('docker logs backend --follow'), stdout=subprocess.PIPE)
    for c in iter(lambda: process.stdout.readline(), b""):
        line = c.decode()
        print(line)

generator()


Config : RwLock { data: Config { defaults: {}, overrides: {}, sources: [], cache: Value { origin: None, kind: Table({"domain": Value { origin: Some("the environment"), kind: String("localhost") }, "smtp_login": Value { origin: Some("the environment"), kind: String("le2filsbertrand@gmail.com") }, "jwt_pass": Value { origin: Some("the environment"), kind: String("pass") }, "ws_port": Value { origin: Some("the environment"), kind: String("2999") }, "smtp_password": Value { origin: Some("the environment"), kind: String("dejurxsjvyiyztrp") }, "ws_exchange_host": Value { origin: Some("config.toml"), kind: String("binance") }, "db_port": Value { origin: Some("config.toml"), kind: I64(5432) }, "db_pass": Value { origin: Some("config.toml"), kind: String("123456789") }, "http_port": Value { origin: Some("the environment"), kind: String("3000") }, "smtp_sender": Value { origin: Some("the environment"), kind: String("le2filsbertrand@gmail.com") }, "debug": Value { origin: Some("the environment"),

KeyboardInterrupt: 

In [18]:
next(stream)

In [ ]:
stream.close()

In [187]:
import threading

threading.enumerate()

[<_MainThread(MainThread, started 16200)>,
 <Thread(IOPub, started daemon 12584)>,
 <Heartbeat(Heartbeat, started daemon 11812)>,
 <ControlThread(Control, started daemon 15548)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 16028)>,
 <ParentPollerWindows(Thread-2, started daemon 9524)>,
 <GarbageCollectorThread(Thread-3, started daemon 16132)>,
 <Thread(Thread-9, started 19496)>,
 <Thread(Thread-10, started 18608)>,
 <Thread(Thread-11, started 14704)>]

In [175]:
import json
import math
import re
import subprocess
from threading import Lock
import threading

conversion_table = {
    "B": 1,
    "KiB" : 2**10,
    "MiB" : 2**20, 
    "GiB" : 2**30,
    "TiB" : 2**40,
    "PiB" : 2**50,
    "kB" : 10**3,
    "MB" : 10**6,
    "GB" : 10**9,
    "TB" : 10**12,
    "PB" : 10**15
}




class DockerCliStats:
    def convert_value(x):
        res = re.findall('([0-9.]+)([a-zA-Z]*)', x)[0]
        return math.floor(float(res[0])* conversion_table[res[1]])

    def parse(x):
        def map_parse(y):
            return {
                **{k:DockerCliStats.convert_value(v.replace(" ", ""))  for v,k in zip(y['BlockIO'].split('/'), ("BlockIO_Input","BlockIO_Output"))},
                **{k:DockerCliStats.convert_value(v.replace(" ", ""))  for v,k in zip(y['MemUsage'].split('/'), ("MemUsage_Usage","MemUsage_Total"))},
                **{k:DockerCliStats.convert_value(v.replace(" ", ""))  for v,k in zip(y['NetIO'].split('/'), ("NetIO_Input","NetIO_Output"))},
                "Name" : y["Name"]
            }
        return list(map(map_parse,x))
        



import shlex
class DockerCli:
    def stream_process(cmd, value, parser=None):
        lock = threading.Lock()
        def generator():
            buffer = []
            process = subprocess.Popen(shlex.split("docker "+ cmd + ' --format "{{ json . }}"'), stdout=subprocess.PIPE)
            for c in iter(lambda: process.stdout.readline(), b""):
                line = c.decode()
                if line.startswith('[2J[H'):
                    if len(buffer) !=0 : yield buffer
                    buffer = []
                    line = line.replace('[2J[H', "")
                
                buffer += [json.loads(line)] if len(line) !=0 else []

        def thread_work():  
            for el in (parser(el) for el in generator()) if parser is not None else generator():
                lock.acquire()
                value[0] = el
                lock.release()

        threading.Thread(target = thread_work).start()


val=[0]
DockerCli.stream_process("stats frontend", val, DockerCliStats.parse)

   


In [182]:
val

[[{'BlockIO_Input': 0,
   'BlockIO_Output': 0,
   'MemUsage_Usage': 36878417,
   'MemUsage_Total': 26865020436,
   'NetIO_Input': 36900,
   'NetIO_Output': 13400,
   'Name': 'frontend'}]]